In [1]:
import os
import numpy as np
import cv2
from datetime import datetime

#Learning
from __future__ import division, print_function, absolute_import
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def shuffle(images,labels):
    idx = np.random.permutation(len(images))
    images = [images[i] for i in idx]
    labels = [labels[i] for i in idx]
    return images,labels

def splitData(nTrain,nLabels):
    test_f = []; train_f = []; test_l = []; train_l = [];

    for j in range(nLabels):
        labels = [0]*(nLabels-1)
        labels.insert(j,1)
        PATH = "BrainTumorProcessedImages/" + str(j+1) + "/"
        nFiles = 100 #len(os.listdir(PATH))
        for i in range(nFiles):
            feature = cv2.imread(PATH+os.listdir(PATH)[i])
            feature = feature - np.mean(feature,axis=0)
            if (i>=nFiles*nTrain):
                test_f.append(feature)
                test_l.append(labels)
            else:
                train_f.append(feature)
                train_l.append(labels)
    
    train_f, train_l = shuffle(train_f,train_l)
    test_f, test_l = shuffle(test_f,test_l)
    return train_f, train_l, test_f, test_l

train_f, train_l, test_f, test_l = splitData(0.8,2)

In [5]:
tf.reset_default_graph() #resetting tf values

#Define AlexNet
def alexnet(model_name,checkpt): 
    network = input_data(shape=[None, 512, 512, 3], name='features') 
    network = conv_2d(network, 96, 11, strides=4, activation='relu') 
    network = max_pool_2d(network, 3, strides=2) 
    network = local_response_normalization(network) 
    network = conv_2d(network, 256, 5, activation='relu') 
    network = max_pool_2d(network, 3, strides=2) 
    network = local_response_normalization(network) 
    network = conv_2d(network, 384, 3, activation='relu') 
    network = conv_2d(network, 384, 3, activation='relu') 
    network = conv_2d(network, 256, 3, activation='relu') 
    network = max_pool_2d(network, 3, strides=2) 
    network = local_response_normalization(network) 
    network = fully_connected(network, 4096, activation='tanh') 
    network = dropout(network, 0.5) 
    network = fully_connected(network, 4096, activation='tanh') 
    network = dropout(network, 0.5) 
    network = fully_connected(network, 2, activation='softmax') 
    network = regression(network, optimizer='momentum', loss='categorical_crossentropy', learning_rate=0.001, name='labels')

    model = tflearn.DNN(network, checkpoint_path = '/tmp/original_alexnet/'+model_name, best_checkpoint_path = '/tmp/original_alexnet/'+model_name,
                max_checkpoints=1, tensorboard_verbose=0)
    
    model_checkpt = 'model/'+model_name+'-'+ str(checkpt) 
    if os.path.exists(model_checkpt+'.meta'):
        model.load(model_checkpt)
        print("Starting from Training Step: " + str(checkpt))
    
    return model

model_name = 'tumors_'+str(datetime.now())
model = alexnet(model_name,0)   

model.fit(train_f, train_l, n_epoch=100, validation_set=(test_f, test_l), shuffle=False,
    show_metric=True, batch_size=64, snapshot_step=100, snapshot_epoch=True, run_id=model_name)

Training Step: 226  | total loss: 0.46882 | time: 69.752s
| Momentum | epoch: 076 | loss: 0.46882 - acc: 0.7707 -- iter: 064/160


KeyboardInterrupt: 